In [ ]:
import requests
import json
import time
import random
from typing import List, Dict, Any

# ==============================================================================
# 1. CONFIGURAÇÃO E CONSTANTES
# ==============================================================================

# --- Configurações da API da NASA ---
NASA_API_BASE_URL = "https://power.larc.nasa.gov/api/temporal/climatology/point"
NASA_API_COMMUNITY = "RE"
NASA_API_START_YEAR = "2010"
NASA_API_END_YEAR = "2024"

# --- Parâmetros Tecnológicos (Apenas para o cálculo de kWh/m²) ---
# Solar
SOLAR_LOSS_FACTOR: float = 0.80 # Perdas do sistema (inversor, cabos, sujeira)

# Eólico
TURBINE_SWEPT_AREA_M2: float = 14314.0 # Área de varredura de uma turbina de referência (m²)
TURBINE_CP: float = 0.45               # Coeficiente de potência da turbina de referência

### REMOVIDO: Constantes relacionadas ao Fator de Capacidade foram removidas para simplificar.

# --- Constantes de Cálculo ---
MONTHS = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"]
DAYS_IN_MONTH: Dict[str, int] = dict(zip(MONTHS, [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]))

# --- Mapeamento de Parâmetros da API ---
# Nota: Nem todos os parâmetros são usados nos cálculos, mas são mantidos para fornecer
# dados brutos contextuais na resposta final (ex: ângulo do painel, direção do vento).
API_PARAMETERS_MAP: Dict[str, str] = {
    "SolarIrradianceOptimal": "SI_TILTED_AVG_OPTIMAL",
    "SolarIrradianceOptimalAngle": "SI_TILTED_AVG_OPTIMAL_ANG",
    "SurfaceAirDensity": "RHOA",
    "WindSpeed50m": "WS50M",
    "WindDirection50m": "WD50M"
}

# ==============================================================================
# 2. CAMADA DE ACESSO À API
# ==============================================================================
def http_get(url: str, params: dict, retries: int = 4, timeout: int = 45) -> Dict[str, Any]:
    """Realiza uma requisição GET HTTP com tratamento de erro robusto e retries."""
    start_time = time.perf_counter()
    for attempt in range(retries):
        try:
            response = requests.get(url, params=params, timeout=timeout)
            response.raise_for_status()
            duration = time.perf_counter() - start_time
            print(f"  [INFO] Requisição para Lat:{params.get('latitude')}, Lon:{params.get('longitude')} bem-sucedida em {duration:.2f}s.")
            return response.json()
        except requests.exceptions.RequestException as e:
            status_code = e.response.status_code if e.response else 'N/A'
            is_retryable = status_code in (429, 500, 502, 503, 504)
            print(f"  [WARN] Falha na requisição (tentativa {attempt + 1}/{retries}): Status {status_code} - {e}")
            if not is_retryable or attempt == retries - 1: raise e
            time.sleep((2 ** attempt) * random.uniform(0.8, 1.2))
    raise RuntimeError("Máximo de tentativas de requisição excedido.")

def fetch_climatology_data(lat: float, lon: float, params_list: List[str]) -> Dict[str, Any]:
    """Busca dados de climatologia da API da NASA POWER para uma única coordenada."""
    payload = {
        "start": NASA_API_START_YEAR, "end": NASA_API_END_YEAR, "latitude": lat,
        "longitude": lon, "community": NASA_API_COMMUNITY, "parameters": ",".join(params_list),
        "units": "metric", "header": "true",
    }
    data = http_get(NASA_API_BASE_URL, params=payload)
    return data.get("properties", {}).get("parameter", {})

# ==============================================================================
# 3. CAMADA DE CÁLCULO
# ==============================================================================

### ALTERAÇÃO: Função simplificada para retornar apenas o dicionário de kWh/m².
def calculate_solar_kwh_per_m2(api_data: Dict[str, Any]) -> Dict[str, float]:
    """Calcula a densidade de energia solar (kWh/m²/mês) para cada mês."""
    insolation_data = api_data.get(API_PARAMETERS_MAP["SolarIrradianceOptimal"], {})
    results = {}
    
    for month in MONTHS:
        daily_insolation = insolation_data.get(month)
        if daily_insolation is None or daily_insolation == -999:
            continue

        days = DAYS_IN_MONTH[month]
        monthly_kwh_per_m2 = daily_insolation * SOLAR_LOSS_FACTOR * days
        results[month] = round(monthly_kwh_per_m2, 2)

    if results:
        results["ANN"] = round(sum(results.values()), 2)

    return results

### ALTERAÇÃO: Função simplificada para retornar apenas o dicionário de kWh/m².
def calculate_wind_kwh_per_m2(api_data: Dict[str, Any]) -> Dict[str, float]:
    """Calcula a densidade de energia eólica (kWh/m²/mês) para cada mês."""
    air_density_data = api_data.get(API_PARAMETERS_MAP["SurfaceAirDensity"], {})
    wind_speed_data = api_data.get(API_PARAMETERS_MAP["WindSpeed50m"], {})
    results = {}
    
    for month in MONTHS:
        air_density = air_density_data.get(month)
        wind_speed = wind_speed_data.get(month)
        if any(v is None or v == -999 for v in [air_density, wind_speed]):
            continue

        hours = DAYS_IN_MONTH[month] * 24
        avg_power_watts = 0.5 * air_density * TURBINE_SWEPT_AREA_M2 * (wind_speed ** 3) * TURBINE_CP
        total_monthly_energy_kwh = (avg_power_watts * hours) / 1000
        
        monthly_kwh_per_m2 = total_monthly_energy_kwh / TURBINE_SWEPT_AREA_M2
        results[month] = round(monthly_kwh_per_m2, 2)

    if results:
        results["ANN"] = round(sum(results.values()), 2)

    return results

# ==============================================================================
# 4. ORQUESTRADOR PRINCIPAL
# ==============================================================================

def run_climate_energy_analysis(locations_payload: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Orquestra a análise de potencial energético para uma lista de locais,
    focando na métrica de densidade de energia (kWh/m²).
    """
    start_time = time.perf_counter()
    analysis_results = []
    failed_locations = []
    param_list = list(API_PARAMETERS_MAP.values())
    
    print("Iniciando análise de potencial energético do clima...")

    for loc in locations_payload:
        city, lat, lon = loc.get("city"), loc.get("lat"), loc.get("lon")
        if not all([city, lat, lon]):
            failed_locations.append({"location": loc, "error": "Formato inválido."})
            continue

        print(f"Processando: {city} (Lat:{lat}, Lon:{lon})")
        
        try:
            api_data = fetch_climatology_data(lat, lon, param_list)
            if not api_data: raise ValueError("Resposta da API vazia.")
            
            solar_kwh_per_m2 = calculate_solar_kwh_per_m2(api_data)
            wind_kwh_per_m2 = calculate_wind_kwh_per_m2(api_data)
            
            raw_metrics = {key: api_data.get(value, {}) for key, value in API_PARAMETERS_MAP.items()}
            
            ### ALTERAÇÃO: Estrutura da resposta final simplificada.
            analysis_results.append({
                "location": {"city": city, "latitude": lat, "longitude": lon},
                "climate_energy_potential": {
                    "solar_kwh_per_m2": solar_kwh_per_m2,
                    "wind_kwh_per_m2": wind_kwh_per_m2
                },
                "raw_nasa_metrics_monthly": raw_metrics
            })
            
        except Exception as e:
            print(f"  [ERROR] Falha ao processar {city}. Erro: {e}")
            failed_locations.append({"location": {"city": city, "latitude": lat, "longitude": lon}, "error": str(e)})

    duration = time.perf_counter() - start_time
    
    return {
        "meta": {
            "timestamp_utc": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
            "processing_time_seconds": round(duration, 2),
            "locations_processed": len(analysis_results),
            "locations_failed": len(failed_locations),
        },
        "data": analysis_results,
        "errors": failed_locations
    }

# ==============================================================================
# 5. BLOCO DE EXECUÇÃO
# ==============================================================================

if __name__ == "__main__":
    locations_payload: List[Dict[str, Any]] = [
        {"city": "Joao Pessoa, PB", "lat": -7.1195, "lon": -34.8451},
        {"city": "Caetite, BA", "lat": -14.0683, "lon": -42.4794},
        {"city": "Sao Miguel do Gostoso, RN", "lat": -5.1225, "lon": -35.6361},
        {"city": "Petrolina, PE", "lat": -9.38, "lon": -40.5},
        {"city": "Munich (Germany)", "lat":  48.13, "lon": 11.57},
        {"city": "Local Inválido", "lat": 999, "lon": 999},
    ]
    
    final_report = run_climate_energy_analysis(locations_payload)
    
    print("\n" + "="*80)
    print("✅ Análise CONCLUÍDA! Resposta final focada em kWh/m²:")
    print("="*80)
    print(json.dumps(final_report, indent=2, ensure_ascii=False))